In [5]:
# import some necessary stuff

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%matplotlib inline
import xarray as xr
import intake
import util
import warnings
from cmip6_preprocessing.preprocessing import read_data

In [10]:
# read all data with volume and oxygen
with warnings.catch_warnings(): # these lines just make sure that the warnings dont clutter your notebook
    warnings.simplefilter("ignore")
    data_dict = read_data(col,
                    experiment_id=['piControl'],
                    grid_label='gn',
                    variable_id=['o2','thkcello', 'areacello'],
                    member_id = 'r1i1p1f1', # so that this runs faster for testing
                    required_variable_id = ['o2', 'thkcello', 'areacello']
                    )

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 2 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 4 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 15 group(s)
CanESM5
NON-REFERENCE MODE. This should just be used for a bunch of variables on the same grid
IPSL-CM6A-LR
NON-REFERENCE MODE. This should just be used for a bunch of variables on the same grid


In [11]:
data_dict['IPSL-CM6A-LR']

<xarray.Dataset>
Dimensions:        (axis_nbounds: 2, lev: 75, member_id: 1, nvertex: 4, time: 500, x: 362, x_right: 362, y: 332, y_right: 332)
Coordinates:
  * time           (time) object 1850-07-02 12:00:00 ... 2349-07-02 12:00:00
  * lev            (lev) float32 0.50576 1.5558553 ... 5698.0605 5902.0576
  * member_id      (member_id) <U8 'r1i1p1f1'
  * x              (x) int64 0 1 2 3 4 5 6 7 ... 354 355 356 357 358 359 360 361
  * x_right        (x_right) float64 0.5 1.5 2.5 3.5 ... 358.5 359.5 360.5 361.5
  * y              (y) int64 0 1 2 3 4 5 6 7 ... 324 325 326 327 328 329 330 331
  * y_right        (y_right) float64 0.5 1.5 2.5 3.5 ... 328.5 329.5 330.5 331.5
    dxe            (y, x_right) float64 1.127e+04 1.127e+04 ... 777.7 777.7
    dyn            (y_right, x) float64 1.136e+04 1.136e+04 ... 1.49e+07
    dxt            (y, x) float64 1.127e+04 1.127e+04 ... 1.71e+03 777.7
    dxne           (y_right, x_right) float64 1.136e+04 1.136e+04 ... 6.022e+03
    dxn            

In [19]:
import numpy as np
from xhistogram.xarray import histogram
def omz_volume(o2, vol, o2_bins, dim=['x', 'y', 'lev']):
    nanmask = np.isnan(o2)
    # Should we also check the bin data for nans?
    full_sum = histogram(
        o2.where(~nanmask),
        bins=[o2_bins],
        weights=vol.where(~nanmask),
        dim=dim,
    )
    return full_sum

In [20]:
o2_bins = np.arange(0,300,10)
ds_test = data_dict['IPSL-CM6A-LR']
vol = ds_test.areacello * ds_test.thkcello

omz_vol = omz_volume(ds_test.o2, vol, o2_bins)

In [ ]:
omz_vol.isel(time=0).plot()